In [ ]:
!pip install gluonts
!pip install --upgrade datasets
!pip install utilsforecast

In [ ]:
!git clone https://github.com/GiuliaGhisolfi/TSFM-ZeroShotEval
%cd TSFM-ZeroShotEval/src

### Import data

In [2]:
import json
with open("data/chronos_data_proprieties.json") as f:
    dataset_properties_map = json.load(f)

In [3]:
DATASET_CHRONOS = ["exchange_rate", "ercot", "dominick"]

### Results file

In [4]:
import csv
import os

# Ensure the output directory exists
output_dir = "results"
os.makedirs(output_dir, exist_ok=True)

# Define the path for the CSV file
csv_file_path = os.path.join(output_dir, "chronos_data_results.csv")

with open(csv_file_path, "w", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(
        [
            "dataset",
            "model",
            "eval_metrics/MSE[mean]",
            "eval_metrics/MSE[0.5]",
            "eval_metrics/MAE[0.5]",
            "eval_metrics/MASE[0.5]",
            "eval_metrics/MAPE[0.5]",
            "eval_metrics/sMAPE[0.5]",
            "eval_metrics/MSIS",
            "eval_metrics/RMSE[mean]",
            "eval_metrics/NRMSE[mean]",
            "eval_metrics/ND[0.5]",
            "eval_metrics/mean_weighted_sum_quantile_loss",
            "domain",
            "num_variates",
        ]
    )

## Evaluation

In [5]:
from gluonts.ev.metrics import (
    MSE,
    MAE,
    MASE,
    MAPE,
    SMAPE,
    MSIS,
    RMSE,
    NRMSE,
    ND,
    MeanWeightedSumQuantileLoss,
)

# Instantiate the metrics
metrics = [
    MSE(forecast_type="mean"),
    MSE(forecast_type=0.5),
    MAE(),
    MASE(),
    MAPE(),
    SMAPE(),
    MSIS(),
    RMSE(),
    NRMSE(),
    ND(),
    MeanWeightedSumQuantileLoss(
        quantile_levels=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    ),
]

### Chronos

In [6]:
model_name = "chronos_bolt_small" # TODO: change to "chronos_t5_base" for the original Chronos model

model_path="amazon/chronos-bolt-small",
# TODO: use "amazon/chronos-t5-base" for the corresponding original Chronos model
# "amazon/chronos-bolt-tiny", "amazon/chronos-bolt-mini", "amazon/chronos-bolt-small", "amazon/chronos-bolt-base",
# "amazon/chronos-t5-tiny", "amazon/chronos-t5-mini", "amazon/chronos-t5-small",
# "amazon/chronos-t5-base", "amazon/chronos-t5-large",

In [ ]:
from utils.chronos_predictor import ChronosPredictor
from gluonts.model import evaluate_model
from gluonts.time_feature import get_seasonality
import pandas as pd


for ds_name in DATASET_CHRONOS:
        print(f"Processing dataset: {ds_name}")
        terms = ["short", "medium", "long"]
        for term in terms:
            ds_config = f"{ds_name}/{term}"

            dataset = pd.read_parquet("data/chronos_benchmark/exchange_rate.arrow")
            season_length = get_seasonality(dataset_properties_map[ds_name]["freq"])
            prediction_length = dataset_properties_map[ds_name]["prediction_length"]

            print(f"Dataset size: {len(dataset.test_data)}")
            model = ChronosPredictor(
                model_path=model_path,
                num_samples=20,
                prediction_length=dataset.prediction_length,
                # Change device_map to "cpu" to run on CPU or "cuda" to run on GPU
                device_map="cpu",
            )
            predictor = model.create_predictor(batch_size=32)

            res = evaluate_model(
                predictor,
                test_data=dataset.test_data,
                metrics=metrics,
                batch_size=512,
                axis=None,
                mask_invalid_label=True,
                allow_nan_forecast=False,
                seasonality=season_length,
            )

            # Append the results to the CSV file
            with open(csv_file_path, "a", newline="") as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(
                    [
                        ds_name,
                        model_name,
                        res["MSE[mean]"][0],
                        res["MSE[0.5]"][0],
                        res["MAE[0.5]"][0],
                        res["MASE[0.5]"][0],
                        res["MAPE[0.5]"][0],
                        res["sMAPE[0.5]"][0],
                        res["MSIS"][0],
                        res["RMSE[mean]"][0],
                        res["NRMSE[mean]"][0],
                        res["ND[0.5]"][0],
                        res["mean_weighted_sum_quantile_loss"][0],
                        ds_name,
                        dataset_properties_map[ds_name]["num_variates"],
                    ]
                )

        print(f"Results for {ds_name} have been written to {csv_file_path}")